<a href="https://colab.research.google.com/github/microprediction/winningnotebooks/blob/main/Luce_Axiom_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers
!pip install winning
!pip install pandas
!pip install scipy

# Luce's Choice Axiom versus the Ability Transform
This notebook considers two different models for inferring preferences. The methodology is as follows. With careful prompt engineering:

*   Ask an LLM to assign probabilities to a set A of tokens
*   Ask an LLM to assign probabilities to a subset $B \subset A$ of tokens

We then try to predict the subset probabilities in two ways:


1.   A simple renormalization
2.   Using an contest model

We then compare the errors.




## A contest model for choice

Since the first method is trivial, let's just implement the second here:

In [189]:
!pip install winning

In [262]:
from winning.std_calibration import std_state_price_implied_ability, STD_UNIT, STD_L, STD_SCALE, std_ability_implied_state_prices
def ability_implied_subrace_probabilities(race:dict, runners:[str])-> dict:
     #   Subrace probabilities
     probs = list(race.values())
     names = list(race.keys())
     abilities = std_state_price_implied_ability(probs, unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     sub_names = [nm for nm in names if nm in runners]
     sub_abil = [a for nm, a in zip(names,abilities) if nm in runners]
     sub_prob = implied_probabilities = std_ability_implied_state_prices(ability=sub_abil,unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     implied = dict( zip(sub_names,sub_prob) )
     return implied


race = {'red':0.5,'green':0.3,'blue':0.2}

runners = ['green','red']
implied = ability_implied_subrace_probabilities(race,runners )
implied

{'red': 0.6169905666139499, 'green': 0.38396120015303187}

# Experiments
Here's the setup...

In [238]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd
import numpy as np

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def fill_in_missing_word(sentence, exclude_words=None, top_k=20):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Find the index of the masked token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get the model's predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    if exclude_words:
        # Get the IDs of the words to exclude
        exclude_ids = tokenizer.convert_tokens_to_ids(exclude_words)
        # Set their logits to a very low value
        logits[0, mask_token_index, exclude_ids] = -float('inf')

    # Apply softmax to get probabilities
    probs = torch.softmax(logits[0, mask_token_index], dim=-1)

    # Get the top_k predictions
    top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)

    # If there's only one mask token, adjust dimensions
    if top_k_indices.dim() == 2 and top_k_indices.size(0) == 1:
        top_k_indices = top_k_indices.squeeze(0)
        top_k_probs = top_k_probs.squeeze(0)

    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_probs = top_k_probs.tolist()

    # Store the top predictions in a dictionary
    predictions = dict(zip(top_k_tokens, top_k_probs))

    return predictions



# Luce Axiom Check
def luce_check(sentence1, sentence2):
    # Get probabilities from both sentences
    probs1 = fill_in_missing_word(sentence1, top_k=100)
    probs2 = fill_in_missing_word(sentence2, top_k=10)

    # Filter out words in sentence2 not present in sentence1
    common_tokens = set(probs1.keys()).intersection(set(probs2.keys()))

    # Create filtered dictionaries with common tokens
    probs1_filtered = {token: probs1[token] for token in common_tokens}
    probs2_filtered = {token: probs2[token] for token in common_tokens}

    # Store the original scores (unnormalized) before renormalization
    original_scores = {token: probs1[token] for token in probs1_filtered}

    # Renormalize probs2 so they sum to 1
    total_prob2 = sum(probs2_filtered.values())
    probs2_normalized = {token: prob / total_prob2 for token, prob in probs2_filtered.items()}

    # Renormalize probs1 so they sum to 1
    total_prob1 = sum(probs1_filtered.values())
    probs1_normalized = {token: prob / total_prob1 for token, prob in probs1_filtered.items()}

    # Also add ability implied ...
    prob2_ability = ability_implied_subrace_probabilities(race=probs1, runners=common_tokens)


    # Merge both into a DataFrame for comparison
    df = pd.DataFrame({
        'Choice': list(probs1_normalized.keys()),
        'Original score': [original_scores[token] for token in probs1_normalized.keys()],
        'Luce score': list(probs1_normalized.values()),
        'Ability score': [ prob2_ability[token] for token in probs1_normalized.keys()],
        'Actual score': [probs2_normalized[token] for token in probs1_normalized.keys()]
    })

    # Add a column for the empirical / Luce ratio
    df['Actual/Luce'] = df['Actual score'] / df['Luce score']
    df['Actual/Ability'] = df['Actual score'] / df['Ability score']
    df['Ability RMSE'] =  np.sqrt(((df['Actual score'] - df['Ability score']) ** 2).mean())
    df['Luce RMSE'] =  np.sqrt(((df['Actual score'] - df['Luce score']) ** 2).mean())
    winner = 'Luce' if df['Luce RMSE'].loc[0]<df['Ability RMSE'].loc[0] else 'Ability'
    df['Winner'] = winner

    df.sort_values('Luce score',inplace=True, ascending=False)

    return df


# Function to display results
def display_predictions(sentence, exclude_words=None):
    predictions = fill_in_missing_word(sentence, exclude_words=exclude_words)
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    print(f"Sentence: {sentence}")
    print("Top predictions:")
    for word, score in sorted_predictions:
        print(f"  {word}: {score:.4f}")
    print("\n")
    return sorted_predictions






/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Eating Choices

In [239]:
sentence1 = f"My favourite type of fruit or vegetable is called a {tokenizer.mask_token} and I eat one of them every day."
sentence2 = f"My favourite type of fruit is called a {tokenizer.mask_token} and I eat one of them every day."
df = luce_check(sentence1, sentence2)
print(df)


       Choice  Original score  Luce score  Ability score  Actual score  \
9      banana        0.114603    0.266236       0.230937      0.178440   
4       mango        0.059991    0.139366       0.134456      0.249007   
2         nut        0.052449    0.121846       0.120143      0.068018   
1       fruit        0.034807    0.080861       0.085385      0.088250   
0       peach        0.032701    0.075967       0.081112      0.077764   
6       berry        0.032125    0.074630       0.079915      0.053912   
3        plum        0.029800    0.069228       0.075029      0.067843   
7       lemon        0.027329    0.063488       0.069837      0.058413   
5  strawberry        0.024800    0.057614       0.064416      0.056728   
8      cherry        0.021852    0.050765       0.057991      0.101623   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
9     0.670235        0.772681      0.046298   0.050724  Ability  
4     1.786718        1.851963      0.046298   0.05

## States

In [240]:
sentence1 = f"My favourite state in the U.S. is {tokenizer.mask_token} and I try to visit once a year."
sentence2 = f"My favourite Western state in the U.S. is {tokenizer.mask_token} and I try to visit once a year."
df = luce_check(sentence1, sentence2)
print(df)

       Choice  Original score  Luce score  Ability score  Actual score  \
6  california        0.108853    0.298702       0.253782      0.201740   
9     arizona        0.067399    0.184951       0.171971      0.100027   
1       texas        0.055019    0.150977       0.145925      0.076750   
8    colorado        0.031841    0.087376       0.093829      0.078146   
7      oregon        0.028873    0.079231       0.086702      0.085660   
5    oklahoma        0.019698    0.054052       0.063678      0.066355   
3      nevada        0.016581    0.045499       0.055441      0.101803   
4     montana        0.015908    0.043653       0.053601      0.142662   
2       idaho        0.010460    0.028704       0.038248      0.049947   
0     wyoming        0.009787    0.026856       0.036224      0.096909   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
6     0.675390        0.794937      0.051792   0.063822  Ability  
9     0.540833        0.581653      0.051792   0.06

## Fear

In [264]:
sentence1 = f"The disease I fear most is {tokenizer.mask_token} and my uncle got it."
sentence2 = f"The infectious disease I fear most is {tokenizer.mask_token} and my uncle got it."
df = luce_check(sentence1, sentence2)
print(df)

       Choice  Original score  Luce score  Ability score  Actual score  \
5      cancer        0.229240    0.470493       0.402558      0.077017   
4     malaria        0.106886    0.219375       0.211679      0.148383   
3        aids        0.053682    0.110178       0.119139      0.166081   
9         hiv        0.032544    0.066793       0.078570      0.398082   
7         flu        0.014566    0.029896       0.040419      0.026580   
8   pneumonia        0.014291    0.029332       0.039776      0.025889   
1    smallpox        0.013489    0.027684       0.037899      0.029321   
6  infectious        0.010419    0.021383       0.030613      0.047714   
2       virus        0.006332    0.012996       0.020290      0.034824   
0       viral        0.005783    0.011870       0.018828      0.046109   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
5     0.163693        0.191318      0.146963   0.165864  Ability  
4     0.676389        0.700978      0.146963   0.16

## Countries

In [265]:
sentence1 = f"My favourite drink is {tokenizer.mask_token} and it tastes great."
sentence2 = f"My favourite alcoholic drink is {tokenizer.mask_token} and it tastes great."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Luce score  Ability score  Actual score  \
6      water        0.087032    0.196648       0.179372      0.148692   
8     coffee        0.079112    0.178752       0.165583      0.075745   
7        gin        0.061759    0.139543       0.134460      0.169855   
1       wine        0.048430    0.109426       0.109678      0.080789   
5        rum        0.034824    0.078685       0.083216      0.134153   
2       beer        0.034671    0.078338       0.082906      0.072892   
4  champagne        0.032639    0.073749       0.078821      0.045811   
9      vodka        0.028924    0.065353       0.071264      0.144282   
0       cola        0.019985    0.045155       0.052329      0.051135   
3    bourbon        0.015203    0.034351       0.041662      0.076646   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
6     0.756133        0.828958      0.046304   0.051566  Ability  
8     0.423745        0.457448      0.046304   0.051566  Abili

## Pets

In [266]:
sentence1 = f"My favorite type of pet or farm animal is a {tokenizer.mask_token}, and it is very loyal."
sentence2 = f"My favorite type of pet is a {tokenizer.mask_token}, and it is very loyal."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Luce score  Ability score  Actual score  \
9        dog        0.111114    0.227469       0.205238      0.170574   
2        cat        0.071628    0.146635       0.141145      0.270100   
5       wolf        0.061467    0.125834       0.123862      0.082584   
6       bear        0.057627    0.117972       0.117255      0.115044   
8       lion        0.044970    0.092062       0.094938      0.127784   
3      tiger        0.040343    0.082590       0.086611      0.041469   
0     rabbit        0.039527    0.080919       0.085133      0.051705   
7  chihuahua        0.033485    0.068549       0.073927      0.033833   
4    leopard        0.017254    0.035322       0.042148      0.068246   
1      mouse        0.011063    0.022648       0.028927      0.038661   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
9     0.749878        0.831106      0.051144   0.051697  Ability  
2     1.841989        1.913636      0.051144   0.051697  Abili

## Furniture

In [267]:
sentence1 = f"I need to buy a new piece of furniture or appliance called a {tokenizer.mask_token} for my house."
sentence2 = f"I need to buy a new piece of furniture called a {tokenizer.mask_token} for my house."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Luce score  Ability score  Actual score  \
2      piano        0.012473    0.219646       0.167629      0.098736   
1       room        0.007054    0.124207       0.117120      0.127745   
5  furniture        0.006571    0.115705       0.111852      0.121403   
9      chair        0.005992    0.105506       0.105533      0.114693   
4    bedroom        0.005213    0.091797       0.096674      0.085581   
3        bed        0.004793    0.084400       0.091601      0.106573   
8       sofa        0.004417    0.077779       0.087060      0.103240   
7      table        0.004067    0.071618       0.082654      0.088269   
6       desk        0.003359    0.059155       0.073166      0.071658   
0    dresser        0.002850    0.050187       0.065851      0.082102   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
2     0.449523        0.589015      0.024375   0.041689  Ability  
1     1.028484        1.090718      0.024375   0.041689  Abili

## Classics

In [268]:
sentence1 = f"They are studying modern or classical {tokenizer.mask_token} at the university."
sentence2 = f"They are studying classical {tokenizer.mask_token} at the university."
df = luce_check(sentence1, sentence2)
print(df)

        Choice  Original score  Luce score  Ability score  Actual score  \
8        music        0.443805    0.573795       0.536867      0.500643   
0    languages        0.146693    0.189659       0.192006      0.100392   
3   literature        0.055413    0.071643       0.078533      0.075768   
7      history        0.049378    0.063840       0.070655      0.026200   
4        dance        0.041666    0.053870       0.060504      0.046529   
5      studies        0.011827    0.015291       0.019256      0.031336   
9  composition        0.010515    0.013594       0.017311      0.018114   
6        greek        0.005779    0.007472       0.010073      0.023592   
1    philology        0.005066    0.006550       0.008941      0.150025   
2       ballet        0.003314    0.004285       0.006095      0.027400   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
8     0.872513        0.932528      0.057072   0.060386  Ability  
0     0.529329        0.522860      0.05

## Big cats

In [269]:
sentence1 = f"My favorite animal at the zoo is the {tokenizer.mask_token}, and I always visit its enclosure."
sentence2 = f"My favorite big cat at the zoo is the {tokenizer.mask_token}, and I always visit its enclosure."
df = luce_check(sentence1, sentence2)
print(df)

    Choice  Original score  Luce score  Ability score  Actual score  \
6     lion        0.219672    0.546050       0.450130      0.204367   
3    tiger        0.060831    0.151210       0.161040      0.201513   
5     bear        0.029336    0.072922       0.090420      0.080488   
4  leopard        0.027689    0.068827       0.086420      0.075231   
2  panther        0.025657    0.063777       0.081401      0.088980   
1      cat        0.025206    0.062657       0.080257      0.294430   
0    mouse        0.013903    0.034558       0.050270      0.054991   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
6     0.374264        0.454018      0.124333   0.157728  Ability  
3     1.332673        1.251321      0.124333   0.157728  Ability  
5     1.103761        0.890162      0.124333   0.157728  Ability  
4     1.093049        0.870529      0.124333   0.157728  Ability  
2     1.395177        1.093105      0.124333   0.157728  Ability  
1     4.699074        3.66859

## Getting Around

In [247]:
sentence1 = f"My preferred mode of transportation is the {tokenizer.mask_token}, it's very convenient."
sentence2 = f"My preferred public transportation is the {tokenizer.mask_token}, it's very convenient."
df = luce_check(sentence1, sentence2)
print(df)

    Choice  Original score  Luce score  Ability score  Actual score  \
0      car        0.167282    0.293049       0.265111      0.186921   
4      bus        0.126271    0.221204       0.207027      0.195267   
6     road        0.057955    0.101527       0.104744      0.046570   
5   subway        0.049472    0.086666       0.091235      0.210164   
2  airport        0.046693    0.081799       0.086772      0.089627   
3  highway        0.044981    0.078798       0.084019      0.054571   
7    train        0.033542    0.058760       0.065048      0.050261   
1    buses        0.019355    0.033907       0.040364      0.073449   
8  freeway        0.018005    0.031542       0.037895      0.046425   
9    metro        0.007277    0.012748       0.017286      0.046745   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
0     0.637848        0.705065      0.051875   0.058197  Ability  
4     0.882744        0.943193      0.051875   0.058197  Ability  
6     0.458697   

## Holidays

In [248]:
sentence1 = f"My favorite holiday is {tokenizer.mask_token}, and I always celebrate it with family."
sentence2 = f"My favorite winter holiday is {tokenizer.mask_token}, and I always celebrate it with family."
df = luce_check(sentence1, sentence2)
print(df[:3])


         Choice  Original score  Luce score  Ability score  Actual score  \
7     christmas        0.803094    0.854978       0.840019      0.545480   
1  thanksgiving        0.086861    0.092472       0.096960      0.132594   
8     halloween        0.021869    0.023282       0.025901      0.030875   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
7     0.638004        0.649366      0.099916    0.10473  Ability  
1     1.433873        1.367515      0.099916    0.10473  Ability  
8     1.326135        1.192054      0.099916    0.10473  Ability  


## Cars

In [251]:
sentence1 = f"The best car brand, in my opinion, is {tokenizer.mask_token}."
sentence2 = f"The best U.K. car brand, in my opinion, is {tokenizer.mask_token}."
df = luce_check(sentence1, sentence2)
print(df)

      Choice  Original score  Luce score  Ability score  Actual score  \
0        bmw        0.084127    0.262735       0.222341      0.316667   
7   mercedes        0.052325    0.163417       0.152603      0.077488   
4       ford        0.047231    0.147506       0.140682      0.126972   
2      honda        0.025577    0.079878       0.086626      0.058192   
1  chevrolet        0.025214    0.078744       0.085659      0.077411   
5    ferrari        0.025131    0.078486       0.085438      0.056851   
3     toyota        0.024701    0.077145       0.084294      0.069140   
9    renault        0.014676    0.045834       0.055937      0.081330   
8     jaguar        0.014036    0.043835       0.053965      0.082484   
6      china        0.007179    0.022420       0.031790      0.053466   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE Winner  
0     1.205270        1.424240      0.043103   0.039289   Luce  
7     0.474172        0.507773      0.043103   0.039289   Luce  
4

## Sports

In [276]:
sentence1 = f"The sport that draws the biggest crowds is {tokenizer.mask_token}."
sentence2 = f"The winter sport that draws the biggest crowds is {tokenizer.mask_token}."
df = luce_check(sentence1, sentence2)
print(df)

       Choice  Original score  Luce score  Ability score  Actual score  \
1    football        0.384491    0.563573       0.516096      0.453847   
3  basketball        0.174353    0.255560       0.253395      0.092775   
6    swimming        0.031364    0.045972       0.055432      0.050431   
7     cycling        0.028256    0.041417       0.050581      0.058380   
5   athletics        0.023070    0.033816       0.042327      0.028875   
4   wrestling        0.016009    0.023465       0.030717      0.033665   
0      rowing        0.012924    0.018943       0.025467      0.027891   
9      hockey        0.008869    0.013001       0.018328      0.050509   
2      skiing        0.001838    0.002694       0.004636      0.157429   
8     skating        0.001064    0.001559       0.002877      0.046197   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
1     0.805304        0.879385      0.074975   0.081484  Ability  
3     0.363027        0.366128      0.074975   0.08

# Programming languages

In [281]:
sentence1 = f"My favorite programming language is {tokenizer.mask_token}, I write code with it daily."
sentence2 = f"My favorite object-oriented programming language is {tokenizer.mask_token}, I write code with it daily."
df = luce_check(sentence1, sentence2)
print(df)
variations = ['functional','object-oriented','procedural','classic']

    Choice  Original score  Luce score  Ability score  Actual score  \
8        c        0.393775    0.519164       0.490528      0.264072   
6     java        0.213168    0.281047       0.277894      0.453514   
1   python        0.082175    0.108342       0.115880      0.133333   
3   pascal        0.025111    0.033108       0.039455      0.093922   
0      php        0.018903    0.024923       0.030531      0.018649   
5    basic        0.011095    0.014628       0.018872      0.007091   
9     ruby        0.007781    0.010259       0.013712      0.015548   
4  algebra        0.002346    0.003093       0.004672      0.003665   
7    swift        0.002103    0.002773       0.004236      0.004351   
2      sql        0.002020    0.002663       0.004086      0.005856   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
8     0.508649        0.538343      0.092565   0.099638  Ability  
6     1.613657        1.631967      0.092565   0.099638  Ability  
1     1.230665   

# Starting the day?

In [282]:
sentence1 = f"I like to drink {tokenizer.mask_token} in the morning to start my day."
sentence2 = f"I like to drink hot {tokenizer.mask_token} in the morning to start my day."
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Original score  Luce score  Ability score  Actual score  \
3  coffee        0.157122    0.549390       0.457351      0.238140   
2   water        0.065113    0.227673       0.237397      0.657931   
1    beer        0.045442    0.158892       0.182155      0.008129   
4     tea        0.008099    0.028318       0.051477      0.059600   
5    milk        0.007799    0.027269       0.050102      0.006568   
0  drinks        0.002419    0.008458       0.021308      0.029632   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
3     0.433463        0.520695       0.20705   0.226047  Ability  
2     2.889812        2.771433       0.20705   0.226047  Ability  
1     0.051160        0.044627       0.20705   0.226047  Ability  
4     2.104672        1.157794       0.20705   0.226047  Ability  
5     0.240859        0.131094       0.20705   0.226047  Ability  
0     3.503310        1.390604       0.20705   0.226047  Ability  


# Scientists

In [255]:
sentence1 = f"My favorite scientist's last name is {tokenizer.mask_token}, they changed the world."
sentence2 = f"My favorite 19th Century scientist's last name is {tokenizer.mask_token}, they changed the world."
df = luce_check(sentence1, sentence2)
print(df)

     Choice  Original score  Luce score  Ability score  Actual score  \
1  einstein        0.022025    0.330548       0.235863      0.106760   
8     james        0.012042    0.180729       0.161632      0.166290   
4     peter        0.010730    0.161033       0.150376      0.116121   
6   charles        0.005971    0.089615       0.104032      0.106001   
5    darwin        0.004373    0.065634       0.085521      0.200846   
7  franklin        0.003387    0.050827       0.072766      0.070512   
0    joseph        0.002859    0.042905       0.065460      0.072710   
2     henry        0.002689    0.040351       0.062895      0.092065   
3    thomas        0.002556    0.038358       0.060893      0.068695   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
1     0.322979        0.452635      0.059755   0.091729  Ability  
8     0.920105        1.028815      0.059755   0.091729  Ability  
4     0.721101        0.772205      0.059755   0.091729  Ability  
6     1.182

# More pets

In [256]:
sentence1 = f"My preferred pet is a {tokenizer.mask_token}, they make great companions."
sentence2 = f"My preferred small pet is a {tokenizer.mask_token}, they make great companions."
df = luce_check(sentence1, sentence2)
print(df)


     Choice  Original score  Luce score  Ability score  Actual score  \
2       cat        0.075033    0.155339       0.147005      0.160443   
5      bear        0.068126    0.141039       0.135448      0.127901   
8      lion        0.062841    0.130099       0.126393      0.090659   
0      deer        0.061382    0.127077       0.123891      0.132946   
6      wolf        0.055966    0.115865       0.114572      0.095933   
3     tiger        0.050032    0.103580       0.104115      0.087945   
9       dog        0.043436    0.089925       0.092329      0.094879   
4  squirrel        0.029307    0.060674       0.066128      0.082661   
1     mouse        0.018558    0.038420       0.044893      0.068674   
7    parrot        0.018347    0.037983       0.044457      0.057961   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
2     1.032855        1.091412      0.017981   0.020622  Ability  
5     0.906848        0.944275      0.017981   0.020622  Ability  
8     

In [257]:
sentence1 = f"My favorite baby name is {tokenizer.mask_token}, and I always associate it with family."
sentence2 = f"My favorite girl's baby name is {tokenizer.mask_token}, and I always associate it with family."
df = luce_check(sentence1, sentence2)
print(df)

     Choice  Original score  Luce score  Ability score  Actual score  \
3      lily        0.015596    0.203468       0.161750      0.145201   
5     emily        0.011547    0.150642       0.133103      0.162862   
0       mia        0.008624    0.112507       0.110161      0.131783   
4     bella        0.008521    0.111161       0.109334      0.088390   
8     sarah        0.006815    0.088915       0.094442      0.094142   
7    rachel        0.005872    0.076603       0.085626      0.071505   
9      kate        0.005852    0.076348       0.085438      0.084366   
1    lauren        0.004680    0.061055       0.073866      0.072298   
6  brittany        0.004630    0.060399       0.073332      0.075525   
2     chloe        0.004515    0.058902       0.072112      0.073929   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
3     0.713629        0.897683      0.015089   0.022656  Ability  
5     1.081119        1.223580      0.015089   0.022656  Ability  
0     

In [260]:
sentence1 = f"My favorite age is {tokenizer.mask_token}, and my daughter is that age"
sentence2 = f"My favorite age for middle school kids is {tokenizer.mask_token}, and my daughter is that age"
df = luce_check(sentence1, sentence2)
print(df)

   Choice  Original score  Luce score  Ability score  Actual score  \
7   eight        0.045801    0.155288       0.142627      0.152468   
9  twelve        0.040842    0.138472       0.130246      0.139279   
4  eleven        0.037958    0.128694       0.123046      0.085072   
0   seven        0.033578    0.113845       0.111671      0.085769   
5    five        0.033333    0.113015       0.111032      0.084129   
3      16        0.027800    0.094257       0.096288      0.103968   
2     six        0.022533    0.076396       0.081655      0.089710   
1      12        0.019562    0.066326       0.073144      0.116064   
6       9        0.018803    0.063752       0.070882      0.068647   
8      13        0.014734    0.049956       0.058531      0.074894   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE   Winner  
7     0.981840        1.068999      0.022921   0.026316  Ability  
9     1.005830        1.069359      0.022921   0.026316  Ability  
4     0.661040        0.6913

# Softmax

Just for fun, we can do some brain surgery on the LLM to force it to obey Luce Choice Axiom.

In [261]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import logging
from transformers import logging as transformers_logging
from transformers import pipeline
transformers_logging.set_verbosity_error()


# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def fill_in_missing_word(sentence, exclude_words=None, top_k=20):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Find the index of the masked token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get the model's predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    if exclude_words:
        # Get the IDs of the words to exclude
        exclude_ids = tokenizer.convert_tokens_to_ids(exclude_words)
        # Set their logits to a very low value
        logits[0, mask_token_index, exclude_ids] = -float('inf')

    # Apply softmax to get probabilities
    probs = torch.softmax(logits[0, mask_token_index], dim=-1).squeeze()

    # Get the top_k predictions
    top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)

    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_probs = top_k_probs.tolist()

    # Store the top predictions in a dictionary
    predictions = dict(zip(top_k_tokens, top_k_probs))

    return predictions

# Use the tokenizer's mask token in the sentence
sentence = f"The space-ship was so {tokenizer.mask_token} it was not possible to see what was behind it."

# First iteration: get original probabilities
original_predictions = fill_in_missing_word(sentence, top_k=20)

# Extract the top predicted word and its probability
first_word = max(original_predictions, key=original_predictions.get)
first_prob = original_predictions[first_word]

print("Original Top Predictions:")
for word, prob in original_predictions.items():
    print(f"{word}: {prob:.6f}")

print(f"\nFirst answer: '{first_word}', Probability: {first_prob:.6f}\n")

# Second iteration: remove the most probable word
new_predictions = fill_in_missing_word(sentence, exclude_words=[first_word], top_k=20)

print("New Top Predictions After Excluding the First Word:")
for word, prob in new_predictions.items():
    print(f"{word}: {prob:.6f}")

# Compare probabilities according to Luce Choice Axiom
print("\nComparing Ratios According to Luce Choice Axiom:\n")

# Calculate the sum of probabilities excluding the first word
sum_original_probs_excl_first = sum(original_predictions[word] for word in original_predictions if word != first_word)
sum_new_probs = sum(new_predictions.values())

print(f"Sum of original probabilities (excluding '{first_word}'): {sum_original_probs_excl_first:.6f}")
print(f"Sum of new probabilities: {sum_new_probs:.6f}")

# Verify that sums are approximately equal (should be close to 1 - first_prob)
print(f"1 - Probability of '{first_word}': {1 - first_prob:.6f}")

# Check ratios for each word in new_predictions
print("\nWord\tOriginal Prob\tExpected New Prob\tActual New Prob\tRatio (Actual/Expected)")
for word in new_predictions:
    original_prob = original_predictions.get(word, 0)
    expected_new_prob = original_prob / (1 - first_prob)
    actual_new_prob = new_predictions[word]
    ratio = actual_new_prob / expected_new_prob if expected_new_prob != 0 else 0
    print(f"{word}\t{original_prob:.6f}\t{expected_new_prob:.6f}\t{actual_new_prob:.6f}\t{ratio:.6f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Original Top Predictions:
large: 0.344735
small: 0.227211
massive: 0.106483
huge: 0.058782
big: 0.030036
close: 0.020969
dark: 0.020564
enormous: 0.017063
low: 0.010211
tiny: 0.010106
heavy: 0.009116
immense: 0.007942
tall: 0.007581
vast: 0.007423
high: 0.006501
narrow: 0.004375
powerful: 0.003378
quiet: 0.003095
thin: 0.003060
silent: 0.002666

First answer: 'large', Probability: 0.344735

New Top Predictions After Excluding the First Word:
small: 0.346745
massive: 0.162502
huge: 0.089707
big: 0.045837
close: 0.032000
dark: 0.031383
enormous: 0.026040
low: 0.015584
tiny: 0.015423
heavy: 0.013912
immense: 0.012120
tall: 0.011569
vast: 0.011328
high: 0.009920
narrow: 0.006677
powerful: 0.005156
quiet: 0.004724
thin: 0.004669
silent: 0.004069
black: 0.003872

Comparing Ratios According to Luce Choice Axiom:

Sum of original probabilities (excluding 'large'): 0.556562
Sum of new probabilities: 0.853237
1 - Probability of 'large': 0.655265

Word	Original Prob	Expected New Prob	Actual New P